In [1]:
import building_tokenizer
reload(building_tokenizer)
import building_tokenizer as toker
import json
import pandas as pd
from collections import Counter, OrderedDict
from copy import deepcopy

In [2]:
buildingName = 'ebu3b'
notUcsdBuildings = ['ghc']

tokenType = 'NoNumber'

if not buildingName in notUcsdBuildings:
    naeDict = dict()
    naeDict['bonner'] = ["607", "608", "609", "557", "610"]
    naeDict['ap_m'] = ['514', '513','604']
    naeDict['bsb'] = ['519', '568', '567', '566', '564', '565']
    naeDict['ebu3b'] = ["505", "506"]
    #naeDict['otterson'] = ["518", "517", "589", "590"]
    naeList = naeDict[buildingName]

    labeledFile = 'metadata/' + buildingName + '_sensor_types_location.csv'
    with open(labeledFile, 'rb') as fp:
        #truthDF = pd.read_excel(fp)
        truthDF = pd.DataFrame.from_csv(fp)
        #truthDF = truthDF.set_index(keys='Unique Identifier')

    wordFeatFile = 'data/wordfeat_'+buildingName+'.pkl'

    tokenTypeList = ['NoNumber', 'Alphanumeric', 'AlphaAndNum', 'NumAsSingleWord']

    bacnetTypeMapDF = pd.DataFrame.from_csv('metadata/bacnettype_mapping.csv')
    unitMap = pd.read_csv('metadata/unit_mapping.csv').set_index('unit')
    for val in Counter(unitMap.keys()).values():
        if val>1:
            "Unit map file ERROR"
            assert(False)
            
            
    trueDF = pd.DataFrame.from_csv('metadata/'+buildingName+'_sensor_types_location.csv')
    sensorDF, nameList, jcinameList, descList, unitList, bacnettypeList, wordList = \
    toker.structure_metadata(buildingName=buildingName, tokenType=tokenType, \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)

    origSensorDF = deepcopy(sensorDF)
    origNameList = deepcopy(nameList)
    origJcinameList = deepcopy(jcinameList)
    origDescList = deepcopy(descList)
    origUnitList = deepcopy(unitList)
    origBacnettypeList = deepcopy(bacnettypeList)
    origWordList = deepcopy(wordList)

    #_, rawNameList, rawJcinameList, rawDescList, _, _, _ = toker.structure_metadata(buildingName=buildingName, tokenType='AlphaAndNum', \
    _, rawNameList, rawJcinameList, rawDescList, _, _, _ =\
    toker.structure_metadata(buildingName=buildingName, tokenType='JustSeparate', \
                             validSrcidList=trueDF.index.tolist(), withDotFlag=False)
    
'''
else: 
    filename = 'metadata/'+buildingName+'_sensor_types_location.csv'
    #   filename = 'metadata/%s_sensor_types_location.csv'%buildingName
    df = pd.read_csv(filename)
    sentenceDict = dict()    
    for i,raw in enumerate(df['bas_raw'].tolist()):
        sentenceDict[i] = toker.tokenize(tokenType, raw)
'''     
    
## Common part
    
#with open('data/'+buildingName+'_str_score_dict.pkl', 'rb') as fp:
#    scoreDictDict = pickle.load(fp)
#    ### If a word is exactly matched with another, fix it.
#    for word, scoreDict in scoreDictDict.items():
#        if word in scoreDict.keys():
#            scoreDictDict[word] = {word:1}
#    scoreDictDict['co'] = {'co2':1}
        
adder = lambda x,y:x+y
splitter = lambda x:x.split()
bacnetTypes = list(set(reduce(adder,map(splitter,origBacnettypeList),[])))
#for bacnetType in bacnetTypes:
#    scoreDictDict[bacnetType] = {bacnetType:1}
#units = list(set(unitMap['word'].tolist()))
#for unit in units:
#    if type(unit)==str:
#        scoreDictDict[unit] = {unit:1}

In [3]:
splitter = lambda x:x.split(' ')
sentenceDict = dict()
for i, row in enumerate(zip(rawNameList, rawJcinameList, rawDescList)):
    para = list()
    for ele in row:
        para.extend([x for x in ele.split(' ') if x!=''])
        para.append('newline')
    sentenceDict[sensorDF.iloc[i].name] = para
with open("metadata/%s_sentence_dict.json"%buildingName, "w") as fp:
    json.dump(sentenceDict, fp)